In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = False  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)


In [4]:
sess = tf.Session(config=config)

tf.keras.backend.set_session(sess)


In [5]:
# tf.keras.backend.set_floatx('float16')
# tf.keras.backend.set_epsilon(1e-3)
print(tf.keras.backend.floatx())
print(tf.keras.backend.epsilon())

float32
1e-07


In [6]:
0

0

In [7]:
from models.resnet import residual_cnn, memtest_cnn, fp16_loss_generator
from data import DataLoader
from models.vanilla import vanilla_cnn
from models.dense import DenseModel
from models.resnet import residual_cnn
from models import create_estimator_from_subclass

In [76]:
data_dir = "D:\\fmri\\ABCD\\data"

dl = DataLoader()

INFO:tensorflow:Loading ROI features...


D:\GitHub\ABCDJJEAN\analysis\cnn\data.py:97: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  skiprows=1),


INFO:tensorflow:done


In [77]:
set(list(dl.vol_labels.dtypes)), dl.vol_labels.shape

({dtype('int64'), dtype('float64')}, (8669, 1570))

In [ ]:
set(list(dl.vol_labels.dtypes)), dl.vol_labels.shape

In [87]:
with open("D:\\fmri\\ABCD\\data\\fmriresults01\\image03\\testing_fns.txt", "r") as f:
    t1 = [s.strip().split("_")[0][0:4] + "_" + s.strip().split("_")[0][4:] for s in f.readlines()]
    
with open("D:\\fmri\\ABCD\\data\\fmriresults01\\image03\\testing2_fns.txt", "r") as f:
    t2 = [s.strip().split("_")[0][0:4] + "_" + s.strip().split("_")[0][4:] for s in f.readlines()]


In [88]:
with open("D://fmri//ABCD//data/jeff_testing_guids.txt", "r") as f:
    test_list = [s.strip() for s in f.readlines()]

In [92]:
set(test_list) - set(t1 + t2)

{'NDAR_INV0GGJJ4H9'}

In [70]:
for idx in np.where(dl.vol_labels.dtypes == 'O')[0]:
    print(idx)
    dl.vol_labels.iloc[:, idx] = dl.vol_labels.iloc[:, idx].apply(float)

1508
1561
1562
1563
1564
1565
1566


In [96]:
set(test_list) - set(dl.vol_labels.index)

{'NDAR_INV0GGJJ4H9'}

In [15]:

tf.reset_default_graph()

features, labels = dl.generator_input_fn(file_list=os.path.join(data_dir, "jeff_overfit_guids.txt"), batch_size=4, repeat=1)
model = DenseModel()

logits = model(features, training=True)

loss = tf.losses.mean_squared_error(labels, logits)
l2 = tf.add_n(model.losses)

loss += l2
optimizer = tf.train.AdamOptimizer(0.001)
with tf.control_dependencies(model.updates):
    grads_and_vars = optimizer.compute_gradients(loss)
    train_op = optimizer.apply_gradients(
                grads_and_vars, global_step=tf.train.get_global_step())

    
with tf.name_scope("gradients"):
    grad_hist_list = [
        tf.summary.histogram(
            v.name.replace(
                ":", "_") + "/gradient", g
        ) for g, v in grads_and_vars]
with tf.name_scope("parameter_hists"):
    histogram_list = [
        tf.summary.histogram(
            var.name.replace(":", "_"), var)
        for var in tf.trainable_variables()]
    
    hist_op = tf.summary.merge([grad_hist_list, histogram_list])  # this should plot gradients?

In [17]:
l2

<tf.Tensor 'AddN:0' shape=() dtype=float32>

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(loss))
    sess.run(train_op)
#     sess.run(hist_op)

3595511.5


In [108]:
tf.reset_default_graph()


In [109]:
features, labels = dl.generator_input_fn(file_list=os.path.join(data_dir, "jeff_overfit_guids.txt"), batch_size=4, repeat=1)
features, labels

({'image': <tf.Tensor 'IteratorGetNext:1' shape=(?, ?, ?, ?, 1) dtype=float32>,
  'volume': <tf.Tensor 'IteratorGetNext:5' shape=(?, 389) dtype=float32>,
  'entropy': <tf.Tensor 'IteratorGetNext:0' shape=(?, 389) dtype=float32>,
  'mean': <tf.Tensor 'IteratorGetNext:3' shape=(?, 389) dtype=float32>,
  'stdev': <tf.Tensor 'IteratorGetNext:4' shape=(?, 389) dtype=float32>},
 <tf.Tensor 'IteratorGetNext:2' shape=(?, 1) dtype=float32>)

In [110]:
tf.reset_default_graph()

features, labels = dl.generator_input_fn(file_list=os.path.join(data_dir, "jeff_overfit_guids.txt"), batch_size=4, repeat=1)

logits2, end_points, l = vanilla_cnn(features)

loss = tf.losses.mean_squared_error(labels, logits2)
optimizer = tf.train.AdamOptimizer(0.001, epsilon=0.1)
grads_and_vars = optimizer.compute_gradients(loss)
train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=tf.train.get_global_step())
grads_and_vars

[(<tf.Tensor 'gradients/model/image/conv3d/Conv3D_grad/tuple/control_dependency_1:0' shape=(15, 15, 15, 1, 16) dtype=float32>,
  <tf.Variable 'model/image/conv3d/kernel:0' shape=(15, 15, 15, 1, 16) dtype=float32>),
 (<tf.Tensor 'gradients/model/image/conv3d/BiasAdd_grad/tuple/control_dependency_1:0' shape=(16,) dtype=float32>,
  <tf.Variable 'model/image/conv3d/bias:0' shape=(16,) dtype=float32>),
 (<tf.Tensor 'gradients/model/image/batch_normalization_v1/batchnorm/mul_grad/tuple/control_dependency_1:0' shape=(16,) dtype=float32>,
  <tf.Variable 'model/image/batch_normalization_v1/gamma:0' shape=(16,) dtype=float32>),
 (<tf.Tensor 'gradients/model/image/batch_normalization_v1/batchnorm/sub_grad/tuple/control_dependency:0' shape=(16,) dtype=float32>,
  <tf.Variable 'model/image/batch_normalization_v1/beta:0' shape=(16,) dtype=float32>),
 (<tf.Tensor 'gradients/model/image/conv3d_1/Conv3D_grad/tuple/control_dependency_1:0' shape=(3, 3, 3, 16, 16) dtype=float32>,
  <tf.Variable 'model/ima

In [152]:
with tf.Session() as sess:
    f = sess.run(features)

In [162]:
with open(os.path.join(data_dir, "jeff_overfit_guids.txt")) as f:
    sid = f.readlines()
sid[0]

'NDAR_INVUY636PFH'

In [165]:
dl.labels.index

Index(['NDAR_INV007W6H7B', 'NDAR_INV00CY2MDM', 'NDAR_INV00HEV6HB',
       'NDAR_INV00LJVZK2', 'NDAR_INV00NPMHND', 'NDAR_INV00X2TBWJ',
       'NDAR_INV014RTM1V', 'NDAR_INV019DXLU4', 'NDAR_INV01EN91PG',
       'NDAR_INV028D3ELL',
       ...
       'NDAR_INVZ5BHJRP7', 'NDAR_INVZ95YA2ZB', 'NDAR_INVZCDF5310',
       'NDAR_INVZEL29M7N', 'NDAR_INVZHCK39Y1', 'NDAR_INVZM2Y9JCA',
       'NDAR_INVZRBMFHKB', 'NDAR_INVZVGAMFG7', 'NDAR_INVZXPAWB32',
       'NDAR_INVZZL0VA2F'],
      dtype='object', name='subject', length=4154)

In [176]:
dl.vol_labels.loc[str(sid[0])].values

array([0.784013, 104.25767, 23.995482, ..., nan, nan, nan], dtype=object)

In [148]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(train_op)
    gv = sess.run(grads_and_vars)
    sess.run(hist_op)

InvalidArgumentError: Nan in summary histogram for: parameter_hists/vanilla_model/problem/kernel_0
	 [[node parameter_hists/vanilla_model/problem/kernel_0 (defined at <ipython-input-147-873a4a50d464>:28) ]]
	 [[node IteratorGetNext (defined at D:\GitHub\ABCDJJEAN\analysis\cnn\data.py:182) ]]

Caused by op 'parameter_hists/vanilla_model/problem/kernel_0', defined at:
  File "C:\tools\miniconda3\envs\miccai-abcd\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\tools\miniconda3\envs\miccai-abcd\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\tools\miniconda3\envs\miccai-abcd\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\tools\miniconda3\envs\miccai-abcd\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\tools\miniconda3\envs\miccai-abcd\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loo

In [87]:

with open("cfgs/vanilla.json", "r") as f:
    cfg = json.load(f)
model_fn, params = create_estimator_from_subclass(VanillaModel, cfg)



In [134]:
[g[0].min() for g in gv]

C:\tools\miniconda3\envs\miccai-abcd\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)


[-0.6541084,
 -0.0015102276,
 -0.2398521,
 -0.0015124897,
 -1.0820638,
 -2.8521754e-08,
 -0.20531039,
 -0.10185695,
 -0.56947744,
 -3.3527613e-07,
 -0.14130549,
 -0.09947908,
 -0.42093903,
 -4.5634806e-08,
 -0.09651737,
 -0.079591446,
 -0.25188792,
 -6.068149e-08,
 -0.13353615,
 -0.0793041,
 -0.5946453,
 -1.4640391e-06,
 -1.0094633,
 -2.2550192,
 nan,
 0.0,
 -1.1919512,
 -3.886716,
 -10.2298765,
 -20.214798]

In [16]:
with tf.Session() as sess:
    sess.run(a)

TypeError: Fetch argument <tensorflow.python.keras.engine.training.Model object at 0x000002821C96EDD8> has invalid type <class 'tensorflow.python.keras.engine.training.Model'>, must be a string or Tensor. (Can not convert a Model into a Tensor or Operation.)

In [31]:
with tf.Session() as sess:
#     print(sess.run(labels))
    print(sess.run(features["volume"]).shape)

(1, 383)


In [12]:
import json
with open("cfgs/dev.json", "r") as f:
    cfg = json.load(f)

In [14]:
model_fn, params = create_estimator_model(residual_cnn, cfg)

In [27]:
e["initial_conv"]._losses

AttributeError: 'Tensor' object has no attribute '_losses'

In [21]:
tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

[]

In [16]:
classifier = tf.estimator.Estimator(
   model_fn=model_fn,
   model_dir="D:\\fmri\\ABCD\\Model",
   params=None
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\fmri\\ABCD\\Model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001570A5A2208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
classifier.train(
    input_fn = lambda: dl.generator_input_fn(file_list=os.path.join(data_dir, "jeff_overfit_guids.txt"), batch_size=4),
    steps=1
)

Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from D:\fmri\ABCD\Model\model.ckpt-0
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into D:\fmri\ABCD\Model\model.ckpt.


InvalidArgumentError: Input to reshape is a tensor with 547560 values, but the requested shape has 567931
	 [[{{node Reshape}}]]
	 [[node IteratorGetNext (defined at D:\GitHub\ABCDJJEAN\analysis\cnn\data.py:162) ]]
	 [[node gradients/model/image/conv3d/Conv3D_grad/Shape (defined at D:\GitHub\ABCDJJEAN\analysis\cnn\models\__init__.py:65) ]]

In [36]:
with tf.Session() as sess:
    print(sess.run(tf.losses.get_regularization_loss()))
    print(sess.run(losses["reg"]))

0.0
0.0


In [34]:
img = np.random.rand(514, 514, 514)
dl.cfg["crop_size"][0]
center = [s // 2 for s in img.shape]
t = img[center[0] - (dl.cfg["crop_size"][0] // 2):center[0] + (dl.cfg["crop_size"][0] // 2) + 1,
 center[1] - dl.cfg["crop_size"][1] // 2:center[1] + dl.cfg["crop_size"][1] // 2 + 1,
 center[2] - dl.cfg["crop_size"][2] // 2:center[2] + dl.cfg["crop_size"][2] // 2 + 1]

In [40]:
257 - 218, 297 - 257

(39, 40)

In [39]:
center[0] - (dl.cfg["crop_size"][0] // 2), center[0] + (dl.cfg["crop_size"][0] // 2) + 1

(218, 297)

In [21]:
# model = vanilla_cnn(dl.cfg["crop_size"] + [1])
model.compile(tf.train.AdamOptimizer(0.005), "MSE")
model.summary()



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 78, 90, 78, 1 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 72, 84, 72, 8 2752        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 72, 84, 72, 8 0           conv3d[0][0]                     
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 72, 84, 72, 8 1736        leaky_re_lu[0][0]      

In [22]:
estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir="D:\\fmri\\ABCD\\output")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\fmri\\ABCD\\output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000228D43E3828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
estimator.train(
    lambda: dl.generator_input_fn(file_list=os.path.join(data_dir, "jeff_training_guids.txt"),
                                  batch_size=32, shuffle_buffer=100, repeat=1),
    hooks=None,
    steps=100,
    max_steps=None,
    saving_listeners=None
)

INFO:tensorflow:Calling model_fn.


KeyError: "The dictionary passed into features does not have the expected inputs keys defined in the keras model.\n\tExpected keys: {'input_1'}\n\tfeatures keys: {'stdev', 'image', 'volume', 'mean', 'entropy'}\n\tDifference: {'input_1', 'image', 'stdev', 'volume', 'mean', 'entropy'}"

In [8]:
custom_loss = fp16_loss_generator(tf.keras.losses.binary_crossentropy, multiplier=1)

In [9]:
gen = create_testing_generator(input_shape)
dataset = tf.data.Dataset.from_generator(gen, 
                                         output_types=(tf.float16, tf.int64), 
                                         output_shapes=(tf.TensorShape(input_shape), tf.TensorShape([])))
dataset = dataset.batch(1)
next_batch = dataset.make_one_shot_iterator().get_next()

# with tf.Session() as sess:
#     a = sess.run(next_batch)

C:\tools\miniconda3\envs\miccai-abcd\lib\site-packages\tensorflow\python\util\tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [10]:
# tf.reset_default_graph()

m = memtest_cnn(input_shape, tf.float16)
# m = residual_cnn(input_shape, tf.float16)
custom_loss = fp16_loss_generator(tf.keras.losses.binary_crossentropy, multiplier=1)
m.compile(tf.train.GradientDescentOptimizer(0.005), custom_loss)

m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 128, 96, 1)    0         
_________________________________________________________________
conv3d (Conv3D)              (None, 82, 114, 82, 32)   108032    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 82, 114, 82, 32)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 82, 114, 82, 32)   351264    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 82, 114, 82, 32)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 41, 57, 41, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 41, 57, 41, 64)    256064    
__________

In [11]:

# with tf.Session(config=config) as sess:
m.fit(dataset, epochs=10, steps_per_epoch=10)

Epoch 1/10
10/10 [==============================] - 76s 8s/step - loss: 0.6889
Epoch 2/10
10/10 [==============================] - 16s 2s/step - loss: 0.6990
Epoch 3/10
10/10 [==============================] - 16s 2s/step - loss: 0.7006
Epoch 4/10
10/10 [==============================] - 16s 2s/step - loss: 0.6937
Epoch 5/10
10/10 [==============================] - 16s 2s/step - loss: 0.6930
Epoch 6/10
10/10 [==============================] - 16s 2s/step - loss: 0.6942
Epoch 7/10
10/10 [==============================] - 16s 2s/step - loss: 0.6949
Epoch 8/10
10/10 [==============================] - 16s 2s/step - loss: 0.6943
Epoch 9/10
10/10 [==============================] - 16s 2s/step - loss: 0.6939
Epoch 10/10
10/10 [==============================] - 16s 2s/step - loss: 0.6937


In [12]:
m.evaluate(dataset, steps=1)

1/1 [==============================] - 1s 959ms/step


575.5

In [13]:
m.predict(dataset, steps=1)

array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float16)

In [14]:
print(tf.VERSION)
print(tf.keras.__version__)

1.12.0
2.1.6-tf


In [15]:
test = next(gen())[0]

In [16]:
m.predict(np.reshape(test, [1] + input_shape))

array([[0.]], dtype=float16)

In [10]:
tf.keras.backend.set_floatx('float32')
tf.keras.backend.set_epsilon(1e-7)

# m = residual_cnn(input_shape, tf.float32)

m = memtest_cnn(input_shape, tf.float32)

m.compile(tf.train.GradientDescentOptimizer(0.05), tf.keras.losses.binary_crossentropy)

m.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 128, 96, 1)    0         
_________________________________________________________________
conv3d (Conv3D)              (None, 82, 114, 82, 32)   108032    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 82, 114, 82, 32)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 82, 114, 82, 32)   351264    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 82, 114, 82, 32)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 41, 57, 41, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 41, 57, 41, 64)    256064    
__________

In [11]:

# with tf.Session(config=config) as sess:
m.fit(dataset, epochs=10, steps_per_epoch=10)

Epoch 1/10
10/10 [==============================] - 57s 6s/step - loss: 6.6731
Epoch 2/10
10/10 [==============================] - 7s 716ms/step - loss: 11.2827
Epoch 3/10
10/10 [==============================] - 7s 717ms/step - loss: 6.4472
Epoch 4/10
10/10 [==============================] - 7s 716ms/step - loss: 9.6709
Epoch 5/10
10/10 [==============================] - 7s 717ms/step - loss: 9.6709
Epoch 6/10
10/10 [==============================] - 7s 718ms/step - loss: 6.4472
Epoch 7/10
10/10 [==============================] - 7s 743ms/step - loss: 11.2827
Epoch 8/10
10/10 [==============================] - 7s 729ms/step - loss: 6.4472
Epoch 9/10
10/10 [==============================] - 7s 716ms/step - loss: 9.6709
Epoch 10/10
10/10 [==============================] - 7s 717ms/step - loss: 6.4472


In [22]:
m.evaluate(dataset, steps=1)

1/1 [==============================] - 1s 1s/step


0.6256953477859497

In [1]:
a = 1